<a href="https://colab.research.google.com/github/kodenshacho/sigma/blob/master/glfragment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#include <GL/glew.h>
#include <GLFW/glfw3.h>
#include <iostream>
#include <vector>
#define STB_IMAGE_IMPLEMENTATION
#include "stb_image.h"

const char* vertexShaderSource = R"glsl(
#version 330 core
layout (location = 0) in vec3 aPos;
layout (location = 1) in vec2 aTexCoord;

out vec2 TexCoord;

void main()
{
    gl_Position = vec4(aPos, 1.0);
    TexCoord = aTexCoord;
}
)glsl";

const char* fragmentShaderSource = R"glsl(
#version 330 core
out vec4 FragColor;

in vec2 TexCoord;

uniform sampler2D texture1;
uniform vec3 colorFilter;

void main()
{
    FragColor = texture(texture1, TexCoord) * vec4(colorFilter, 1.0);
}
)glsl";

float vertices[] = {
    // positions         // texture coords
     0.5f,  0.5f, 0.0f,  1.0f, 1.0f, // top right
     0.5f, -0.5f, 0.0f,  1.0f, 0.0f, // bottom right
    -0.5f, -0.5f, 0.0f,  0.0f, 0.0f, // bottom left
    -0.5f,  0.5f, 0.0f,  0.0f, 1.0f  // top left
};
unsigned int indices[] = {
    0, 1, 3, // first triangle
    1, 2, 3  // second triangle
};

unsigned int VBO, VAO, EBO;
unsigned int shaderProgram;

void setupMesh() {
    // Generate buffers and arrays
    glGenVertexArrays(1, &VAO);
    glGenBuffers(1, &VBO);
    glGenBuffers(1, &EBO);

    glBindVertexArray(VAO);
    glBindBuffer(GL_ARRAY_BUFFER, VBO);
    glBufferData(GL_ARRAY_BUFFER, sizeof(vertices), vertices, GL_STATIC_DRAW);

    glBindBuffer(GL_ELEMENT_ARRAY_BUFFER, EBO);
    glBufferData(GL_ELEMENT_ARRAY_BUFFER, sizeof(indices), indices, GL_STATIC_DRAW);

    // position attribute
    glVertexAttribPointer(0, 3, GL_FLOAT, GL_FALSE, 5 * sizeof(float), (void*)0);
    glEnableVertexAttribArray(0);
    // texture coord attribute
    glVertexAttribPointer(1, 2, GL_FLOAT, GL_FALSE, 5 * sizeof(float), (void*)(3 * sizeof(float)));
    glEnableVertexAttribArray(1);
}

unsigned int loadShader(GLenum type, const char* shaderSource) {
    unsigned int shader;
    shader = glCreateShader(type);
    glShaderSource(shader, 1, &shaderSource, NULL);
    glCompileShader(shader);
    // Check for shader compile errors
    int success;
    char infoLog[512];
    glGetShaderiv(shader, GL_COMPILE_STATUS, &success);
    if (!success) {
        glGetShaderInfoLog(shader, 512, NULL, infoLog);
        std::cout << "ERROR::SHADER::COMPILATION_FAILED\n" << infoLog << std::endl;
    }
    return shader;
}

unsigned int loadTexture(const char* path) {
    unsigned int textureID;
    glGenTextures(1, &textureID);

    int width, height, nrChannels;
    stbi_set_flip_vertically_on_load(true);
    unsigned char* data = stbi_load(path, &width, &height, &nrChannels, 0);
    if (data) {
        GLenum format;
        if (nrChannels == 1)
            format = GL_RED;
        else if (nrChannels == 3)
            format = GL_RGB
        else if (nrChannels == 4)
            format = GL_RGBA;

        glBindTexture(GL_TEXTURE_2D, textureID);
        glTexImage2D(GL_TEXTURE_2D, 0, format, width, height, 0, format, GL_UNSIGNED_BYTE, data);
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT);
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT);
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR);
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR);

        stbi_image_free(data);
    } else {
        std::cout << "Failed to load texture" << std::endl;
    }

    return textureID;
}

void setupShader() {
    unsigned int vertexShader, fragmentShader;
    vertexShader = loadShader(GL_VERTEX_SHADER, vertexShaderSource);
    fragmentShader = loadShader(GL_FRAGMENT_SHADER, fragmentShaderSource);

    shaderProgram = glCreateProgram();
    glAttachShader(shaderProgram, vertexShader);
    glAttachShader(shaderProgram, fragmentShader);
    glLinkProgram(shaderProgram);

    // Check for linking errors
    int success;
    char infoLog[512];
    glGetProgramiv(shaderProgram, GL_LINK_STATUS, &success);
    if (!success) {
        glGetProgramInfoLog(shaderProgram, 512, NULL, infoLog);
        std::cout << "ERROR::SHADER::PROGRAM::LINKING_FAILED\n" << infoLog << std::endl;
    }
    glDeleteShader(vertexShader);
    glDeleteShader(fragmentShader);
}

void renderLoop(GLFWwindow* window, std::vector<unsigned int>& textures, std::vector<glm::vec3>& colorFilters) {
    while (!glfwWindowShouldClose(window)) {
        glClearColor(0.2f, 0.3f, 0.3f, 1.0f);
        glClear(GL_COLOR_BUFFER_BIT);

        glUseProgram(shaderProgram);

        for (size_t i = 0; i < textures.size(); ++i) {
            glActiveTexture(GL_TEXTURE0);
            glBindTexture(GL_TEXTURE_2D, textures[i]);
            glUniform1i(glGetUniformLocation(shaderProgram, "texture1"), 0);

            glUniform3f(glGetUniformLocation(shaderProgram, "colorFilter"), colorFilters[i].x, colorFilters[i].y, colorFilters[i].z);

            glBindVertexArray(VAO);
            glDrawElements(GL_TRIANGLES, 6, GL_UNSIGNED_INT, 0);
        }

        glfwSwapBuffers(window);
        glfwPollEvents();
    }
}

int main() {
    glfwInit();
    glfwWindowHint(GLFW_CONTEXT_VERSION_MAJOR, 3);
    glfwWindowHint(GLFW_CONTEXT_VERSION_MINOR, 3);
    glfwWindowHint(GLFW_OPENGL_PROFILE, GLFW_OPENGL_CORE_PROFILE);

    GLFWwindow* window = glfwCreateWindow(800, 600, "OpenGL Example", NULL, NULL);
    if (window == NULL) {
        std::cout << "Failed to create GLFW window" << std::endl;
        glfwTerminate();
        return -1;
    }
    glfwMakeContextCurrent(window);

    if (glewInit() != GLEW_OK) {
        std::cout << "Failed to initialize GLEW" << std::endl;
        return -1;
    }

    setupMesh();
    setupShader();

    std::vector<unsigned int> textures;
    std::vector<glm::vec3> colorFilters = {
        glm::vec3(1.0f, 0.0f, 0.0f), // Red
        glm::vec3(0.0f, 1.0f, 0.0f), // Green
        glm::vec3(0.0f, 0.0f, 1.0f), // Blue
        glm::vec3(0.0f, 1.0f, 1.0f), // Cyan
        glm::vec3(1.0f, 0.0f, 1.0f), // Magenta
        glm::vec3(1.0f, 1.0f, 0.0f)  // Yellow
    };

    // 이미지 경로는 실제 경로로 대체해야 합니다.
    for (int i = 0; i < 6; ++i) {
        textures.push_back(loadTexture("path_to_your_image"));
    }

    renderLoop(window, textures, colorFilters);

    glfwTerminate();
    return 0;
}

In [ ]:

!!ARBfp1.0

PARAM colorInclude = program.local[0]; // 색상 포함 여부를 나타내는 파라미터
TEMP textureColor; // 텍스처의 색상을 저장할 임시 변수

# 텍스처에서 색상을 가져옵니다.
TEX textureColor, fragment.texcoord[0], texture[0], 2D;

# 색상 포함 여부에 따라 텍스처의 색상을 조정합니다.
MUL result.color, textureColor, colorInclude;

END

In [ ]:

!!ARBfp1.0

PARAM activation = program.local[0]; // 各テクスチャの活性化状態
TEMP texColor[6]; // テクスチャ色を保存するための一時変数配列
TEMP activeCount; // 活性化されたテクスチャの数
TEMP finalColor; // 最終色

# 初期化
MOV activeCount, {0.0, 0.0, 0.0, 0.0};
MOV finalColor, {0.0, 0.0, 0.0, 0.0};

# 6つのテクスチャを順番に処理し、活性化状態に応じて色を合算
FOR i = 0; i < 6; i = i + 1 DO
    TEX texColor[i], fragment.texcoord[i], texture[i], 2D;
    MUL texColor[i], texColor[i], activation[i];
    ADD finalColor, finalColor, texColor[i];
    ADD activeCount, activeCount, activation[i];
ENDFOR

# 活性化されたテクスチャの平均色を計算
DIV finalColor, finalColor, activeCount;

# 結果を適用
MOV result.color, finalColor;

END

#include <GL/glew.h>
#include <GLFW/glfw3.h>

void setupFragmentProgram() {
    GLuint fragProgram;
    glGenProgramsARB(1, &fragProgram);
    glBindProgramARB(GL_FRAGMENT_PROGRAM_ARB, fragProgram);

    // フラグメントプログラムのソースをロードしてコンパイル（ソースコードは文字列変数に保存されていると仮定）
    glProgramStringARB(GL_FRAGMENT_PROGRAM_ARB, GL_PROGRAM_FORMAT_ASCII_ARB,
                       strlen(fragProgramSource), fragProgramSource);

    // 各テクスチャの活性化状態を設定します。例：最初と三番目のテクスチャのみを活性化
    // 活性化されたテクスチャには1.0を、非活性化されたテクスチャには0.0を設定します。
    GLfloat activation[4] = {1.0, 0.0, 1.0, 0.0, 0.0, 0.0}; // 6つ中4つのみ設定例
    glProgramLocalParameter4fvARB(GL_FRAGMENT_PROGRAM_ARB, 0, activation);
}

// レンダリングループおよびその他のコードは省略

In [ ]:

!!ARBfp1.0

PARAM colorEnabled = program.local[0]; // 색상 활성화 상태 (red, green, blue, cyan, magenta, yellow)
TEMP texColor; // 텍스처 색상

# 텍스처 색상 가져오기
TEX texColor, fragment.texcoord[0], texture[0], 2D;

# 색상 활성화 여부에 따라 필터링
# red
MUL texColor.r, texColor.r, colorEnabled.r;
# green
MUL texColor.g, texColor.g, colorEnabled.g;
# blue
MUL texColor.b, texColor.b, colorEnabled.b;

# cyan, magenta, yellow 필터링
# cyan = green + blue
MUL texColor.c, texColor.g, colorEnabled.a;
MUL texColor.c, texColor.b, colorEnabled.a;
ADD texColor.c, texColor.g, texColor.b;

# magenta = red + blue
MUL texColor.m, texColor.r, colorEnabled.a;
MUL texColor.m, texColor.b, colorEnabled.a;
ADD texColor.m, texColor.r, texColor.b;

# yellow = red + green
MUL texColor.y, texColor.r, colorEnabled.a;
MUL texColor.y, texColor.g, colorEnabled.a;
ADD texColor.y, texColor.r, texColor.g;

# 결과 적용
MOV result.color, texColor;

END

#include <GL/glew.h>
#include <GLFW/glfw3.h>

void setupFragmentProgram() {
    GLuint fragProgram;
    glGenProgramsARB(1, &fragProgram);
    glBindProgramARB(GL_FRAGMENT_PROGRAM_ARB, fragProgram);

    // 프래그먼트 프로그램 소스 로드 및 컴파일 (소스 코드는 문자열 변수에 저장되어 있다고 가정)
    const char* fragProgramSource = ...; // 프래그먼트 프로그램 소스
    glProgramStringARB(GL_FRAGMENT_PROGRAM_ARB, GL_PROGRAM_FORMAT_ASCII_ARB,
                       strlen(fragProgramSource), fragProgramSource);

    // 색상 활성화 상태를 설정합니다. 예: red, green, blue 만 활성화
    // 활성화된 색상에는 1.0을, 비활성화된 색상에는 0.0을 설정합니다.
    GLfloat colorEnabled[4] = {1.0, 1.0, 1.0, 0.0}; // cyan, magenta, yellow는 비활성화
    glProgramLocalParameter4fvARB(GL_FRAGMENT_PROGRAM_ARB, 0, colorEnabled);
}

// 렌더링 루프 및 기타 코드는 생략